In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

matplotlib.rcParams['figure.figsize'] = (30, 30)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

In [ ]:
# Pre-process for data in US 
dfUS = pd.read_csv('/kaggle/input/trending-youtube-videos-by-category/21.25.06_VN_videos.csv', index_col='video_id')
dfUS.dataframeName = 'USvideos.csv'
nRow, nCol = dfUS.shape

dfUS['trending_date'] = pd.to_datetime(dfUS['trending_date'], format='%y.%d.%m')
dfUS.insert(0, 'trending_year', dfUS['trending_date'].dt.year)
dfUS.insert(0, 'trending_month', dfUS['trending_date'].dt.month)


# Process data
type_int_list = ['view_count', 'likes', 'dislikes', 'comment_count']
for column in type_int_list:
    dfUS[column] = dfUS[column].astype(int)

type_str_list = ['categoryId']
for column in type_str_list:
    dfUS[column] = dfUS[column].astype(str)
    
# creates a dictionary that maps `category_id` to `category`
id_to_category = {}

with open('/kaggle/input/trending-youtube-videos-by-category/videoCategories.json', 'r') as f:
    data = json.load(f)
    for category in data['items']:
        id_to_category[category['id']] = category['snippet']['title']
        
dfUS.insert(4, 'category', dfUS['categoryId'].map(id_to_category))
dfUS[['categoryId', 'category']].head()

category_count = dfUS['category'].value_counts() # frequency for each category
category_count

ax = category_count.plot.pie(autopct='%1.1f%%',  shadow=True, startangle=0, fontsize=15)

In [ ]:
# Trend  của năm 2019 có  khác  trend  năm 2020 (2021) không? Sau  đại dịch thì thể loại trending  video thay đổi thế nào?
sumCatUS_Date = dfUS.groupby(['trending_year', 'category']).sum()[['view_count']]
sumCatUS_Date

colors = ['black', 'red', 'chocolate', 'orange', 'pink', 'olive', 'yellow', 'green', 'teal', 'dodgerblue', 'blue', 'blueviolet', 'purple', 'deeppink','crimson', 'grey', 'coral']
table = pd.pivot_table(sumCatUS_Date, index='trending_year', columns='category', values='view_count')
ax = table.plot(kind='bar', color = colors)
ax.set_xticklabels(labels=table.index, rotation=45, fontsize=15)

plt.show()

In [ ]:
# Thể  loại  video  nào  tạo  được nhiều tương tác hơn với người xem? (có  nhiều  commentt/reaction)

sumCatUS = dfUS.groupby(['category']).sum()[['comment_count']]

ax = sumCatUS.plot(kind='bar')
ax.set_xticklabels(labels=sumCatUS.index, rotation=45, fontsize=15)

plt.show()

In [ ]:
# "Thể  loại video trending thay đổi theo  từng  tháng  như  thế  nào? -> có  thể  suy  ra  tháng  đó  có  eventt/occasion  gì đặc biệt"

sumCatUSMonth2017 = dfUS[dfUS.trending_year.eq(2021)].groupby([ 'trending_year', 'trending_month','category']).sum()[['view_count']]

table = pd.pivot_table(sumCatUSMonth2017, index='trending_month', columns='category', values='view_count')
ax = table.plot(kind='bar', color = colors)
ax.set_xticklabels(labels=table.index, rotation=45, fontsize=15)

plt.show()

In [ ]:
sumCatUSMonth2018 = dfUS[dfUS.trending_year.eq(2018)].groupby(['category', 'trending_year', 'trending_month']).sum()[['views','likes', 'dislikes', 'comment_count']]
sumCatUSMonth2018

ax = sumCatUSMonth2018.plot.bar()
ax.set_xticklabels(labels=sumCatUSMonth2018.index, rotation=45, fontsize=10)

plt.show()